# CAO Points

## Introduction

## Acquiring the data

In [91]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
from tabula import read_pdf
import requests


In [92]:
# 2019 data is provided in PDF format
cao_2019_l8_url = 'http://www2.cao.ie/points/lvl8_19.pdf'
cao_2019_l76_url = 'http://www2.cao.ie/points/lvl76_19.pdf'

# read the entire pdf, extracting tables into a single dataframe
df = read_pdf("lvl8_19.pdf",pages="all",multiple_tables=False)[0]

type(df)

pandas.core.frame.DataFrame

## Analysing the data

pandas.core.frame.DataFrame

In [90]:
df['Course Code']

0        NaN
1      AL801
2      AL802
3      AL803
4      AL805
       ...  
960    WD200
961    WD210
962    WD211
963    WD212
964    WD230
Name: Course Code, Length: 965, dtype: object

## Conclusion

## References